In [10]:
from pyspark.sql import SparkSession
import os

minio_endpoint   = os.getenv("MINIO_ENDPOINT", "http://minio:9000")
minio_access_key = os.getenv("MINIO_ROOT_USER", "admin")
minio_secret_key = os.getenv("MINIO_ROOT_PASSWORD", "password")

spark = (
    SparkSession.builder
    .appName("Spark Running Test")
    .master("spark://spark-iceberg:7077")
    .master("spark://spark-iceberg:7077") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.demo.type", "rest") \
    .config("spark.sql.catalog.demo.uri", "http://rest:8181") \
    .config("spark.sql.catalog.demo.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.demo.warehouse", "s3://warehouse/wh/") \
    .config("spark.sql.catalog.demo.s3.endpoint", "http://minio:9000") \
    .config("spark.sql.defaultCatalog", "demo") \
    .getOrCreate()
)

In [11]:
spark

In [13]:
%%sql

CREATE DATABASE test

25/09/20 13:16:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [16]:
%%sql

SHOW DATABASES

namespace
test


In [12]:
df = spark.read.option("inferSchema","true").option("multiline","true").json("/home/iceberg/data/nyc_film_permits.json")
df.show()

+---------+-----------------+----------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+----------------+--------------------+------------------+--------------------+
|  borough|         category|communityboard_s|             country|         enddatetime|           enteredon|         eventagency|eventid|           eventtype|         parkingheld|policeprecinct_s|       startdatetime|   subcategoryname|           zipcode_s|
+---------+-----------------+----------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+--------------------+----------------+--------------------+------------------+--------------------+
|Manhattan|       Television|               5|United States of ...|2025-06-20T13:00:...|2025-06-17T05:01:...|Mayor's Office of...| 870952|     Shooting Permit|WEST 47 STREET be...|              18|2025-06-20T01:00:...|     

In [18]:
schema_name = 'test'
table_name = 'test_table'

try:
    df.write.mode("overwrite").saveAsTable(f"{schema_name}.{table_name}")
    print(f"Data successfully written to Iceberg table: {schema_name}.{table_name}")
except Exception as e:
    print(f"Failed to write data to Iceberg table: {schema_name}.{table_name}", exc_info=True)
    raise

25/09/20 13:20:42 WARN TaskSetManager: Lost task 0.0 in stage 2.0 (TID 2) (172.19.0.6 executor 0): software.amazon.awssdk.core.exception.SdkClientException: Unable to load region from any of the providers in the chain software.amazon.awssdk.regions.providers.DefaultAwsRegionProviderChain@2c64f5f8: [software.amazon.awssdk.regions.providers.SystemSettingsRegionProvider@170f925: Unable to load region from system settings. Region must be specified either via environment variable (AWS_REGION) or  system property (aws.region)., software.amazon.awssdk.regions.providers.AwsProfileRegionProvider@348e6d2b: No region provided in profile: default, software.amazon.awssdk.regions.providers.InstanceProfileRegionProvider@4ffac37e: Unable to contact EC2 metadata service.]
	at software.amazon.awssdk.core.exception.SdkClientException$BuilderImpl.build(SdkClientException.java:111)
	at software.amazon.awssdk.regions.providers.AwsRegionProviderChain.getRegion(AwsRegionProviderChain.java:70)
	at software.ama

TypeError: 'exc_info' is an invalid keyword argument for print()

In [19]:
%%sql

SELECT * FROM test.test_table

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `test`.`test_table` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 2 pos 14;
'Project [*]
+- 'UnresolvedRelation [test, test_table], [], false


In [8]:
spark.stop()